# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader 
from tqdm import tqdm
from torch import optim
from torch.autograd import Variable
import time


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
NUM_EPOCHS = 10
learning_rate=0.001

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [5]:

class SimpelNet(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, num_classes):
        super(SimpelNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.relu = nn.LeakyReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

model = SimpelNet(28*28, 100,100, 10).to(device)



# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=0.9)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.8) #比sgd多了一个momentum参数
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99))


Next, we can start to train and evaluate!

In [6]:
# train and evaluate
total_step = len(train_loader)
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # 前向传播和计算loss
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 后向传播和调整参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 每100个batch打印一次数据
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, NUM_EPOCHS, i+1, total_step, loss.item()))
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))




            
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
  
    
    


Epoch [1/10], Step [100/468], Loss: 0.4900
Epoch [1/10], Step [200/468], Loss: 0.3363
Epoch [1/10], Step [300/468], Loss: 0.3171
Epoch [1/10], Step [400/468], Loss: 0.2928
Accuracy of the network on the test images: 92.40785256410257 %
Epoch [2/10], Step [100/468], Loss: 0.2862
Epoch [2/10], Step [200/468], Loss: 0.2734
Epoch [2/10], Step [300/468], Loss: 0.2379
Epoch [2/10], Step [400/468], Loss: 0.1573
Accuracy of the network on the test images: 95.3525641025641 %
Epoch [3/10], Step [100/468], Loss: 0.1918
Epoch [3/10], Step [200/468], Loss: 0.2898
Epoch [3/10], Step [300/468], Loss: 0.1567
Epoch [3/10], Step [400/468], Loss: 0.0791
Accuracy of the network on the test images: 96.70472756410257 %
Epoch [4/10], Step [100/468], Loss: 0.0718
Epoch [4/10], Step [200/468], Loss: 0.1112
Epoch [4/10], Step [300/468], Loss: 0.0822
Epoch [4/10], Step [400/468], Loss: 0.1764
Accuracy of the network on the test images: 96.38421474358974 %
Epoch [5/10], Step [100/468], Loss: 0.1278
Epoch [5/10], 

#### Q5:
Please print the training and testing accuracy.

In [7]:
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 97.59615384615384 %
